In [6]:
from dotenv import load_dotenv
from langgraph.graph import StateGraph,END,add_messages
from langgraph.types import Command,interrupt
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict,Annotated
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",
)
memory = MemorySaver()


In [ ]:
class AgentState(TypedDict):
    message: str
def node_a(state: AgentState):
    print("Inside node a")
    return Command(
        goto="node_b",
        update={
            "message":state["message"]+"a"
        }
    )

def node_b(state: AgentState):
    print("\ninside node b\t")
    human_response = interrupt("you want to go node c or d?")
    print(f'human response: {human_response}')
    if(human_response=="c"):
        return Command(
            goto="node_c",
            update={
                "message":state["message"]+"b"
            }
        )
    else:
        return Command(
        goto="node_c",
        update={
            "message":state["message"]+"b"
        }
            )


def node_c(state:AgentState):
    print(f"inside node c")
    return Command(
        goto=END,
        update={
            "message":state["message"]+"c"
        }
    )
def node_d(state: AgentState):
    print("inside node d")
    return Command(
        goto=END,
        update={
            "message":state["message"]+"d"
        }
    )


In [20]:
graph = StateGraph(AgentState)

graph.add_node("node_a",node_a)
graph.add_node("node_b",node_b)
graph.add_node('node_c',node_c)
graph.add_node("node_d",node_d)
graph.set_entry_point("node_a")
app = graph.compile(checkpointer=memory)
config = {
    "configurable":{"thread_id":1}
}
result_1 = app.invoke({
        "message":""
    },config=config,stream_mode="updates")
result_1

Inside node a

inside node b	


[{'node_a': {'message': 'a'}},
 {'__interrupt__': (Interrupt(value='you want to go node c or d?', resumable=True, ns=['node_b:cf113922-9fd3-19b3-71c3-09179840c76a']),)}]

In [21]:
print(app.get_state(config).next)

('node_b',)


In [23]:
result_2 = app.invoke(Command(resume="c"),config=config,stream_mode="updates")
result_2

[]